In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/daconsets2/train.csv")
test = pd.read_csv("../input/daconsets2/test.csv")
display(train,test)

In [ ]:
all_data = pd.concat([train,test]).reset_index(drop=True)
all_data

In [ ]:
new_df = all_data.iloc[:,0:7]
new_df

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
new_df["요일"] = le.fit_transform(new_df["요일"])

In [ ]:
new_df = pd.get_dummies(new_df.iloc[:,1:7])
new_df

In [ ]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer() # 너무 많이 등장하지 않았거나 등장한 단어 제거할 수 있다. 옵션을 통해
tk.fit_on_texts(all_data["중식메뉴"])
#tk.word_index

In [ ]:
tk.word_index["잡곡밥"] # 특수 문자가 자동으로 무시됨, tk의 filter라는 옵션이 특수 문자 무시함, 다른 특수문자가 중요한 대회에서는 중요할 수도 있다.(댓글 분석, 감정 표현, 욕, 이모티콘)

In [ ]:
tk.word_counts["잡곡밥"] # 등장한 단어 횟수

In [ ]:
text = tk.texts_to_sequences(all_data["중식메뉴"]) 

In [ ]:
from keras.preprocessing.sequence import pad_sequences
pad_text = pad_sequences(text)
pad_text # 토큰나이저가 1부터 시작

In [ ]:
pad_text.shape 

In [ ]:
tk2 = Tokenizer()
tk2.fit_on_texts(all_data["석식메뉴"])

In [ ]:
text2 = tk2.texts_to_sequences(all_data["석식메뉴"]) 

In [ ]:
pad_text2 = pad_sequences(text2)
pad_text2

In [ ]:
pad_text2.shape

In [ ]:
new_train = new_df[:len(train)]
new_test = new_df[len(train):]

In [ ]:
train_text = pad_text[:len(train)]
test_text = pad_text[len(train):]

In [ ]:
train_text2 = pad_text2[:len(train)]
test_text2 = pad_text2[len(train):]

In [ ]:
len(tk.word_index)

In [ ]:
# 중식계
result = 0
best_score = 0
from sklearn.model_selection import KFold
from keras import *
from keras.layers import *
from keras.callbacks import *
i = 0
kf = KFold(n_splits = 5, shuffle = True, random_state = 82) 
for train_index, valid_index in kf.split(train_text):
    i += 1
    print(f"{i}회")
    es = EarlyStopping(patience = 20, verbose = 1) 
    mc = ModelCheckpoint(f"best{i}.h5",save_best_only = True, verbose = 1)
    X_train, X_valid = train_text[train_index], train_text[valid_index] 
    X_train2,X_valid2 = new_train.iloc[train_index], new_train.iloc[valid_index]
    y_train, y_valid = train["중식계"].iloc[train_index], train["중식계"].iloc[valid_index]
    inp1 = Input((6)) 
    inp2 = Input((22))
    
    num_layer = Dense(32,activation = "elu")(inp1) 
    emb = Embedding(1790,10)(inp2) 
    
    branch1 = Dense(16,activation = "elu")(num_layer)
    branch2 = Flatten()(emb)
    
    concat = concatenate([branch1, branch2])
    output = Dense(1)(concat)
    model = Model([inp1,inp2],output)
    
    model.compile(metrics=["mae"], optimizer = "adam", loss = "mse")
    history = model.fit([X_train2,X_train],y_train,validation_data = ([X_valid2,X_valid],y_valid), epochs = 500, callbacks = [es,mc]) # 내부 베스트 점수 보기위한 history
    # rl은 이미지나 오디오에서 많이 쓰인다. 텍스트에서는 잘 쓰이지 않는다.
    model.load_weights(f"best{i}.h5")
    result += model.predict([new_test,test_text], verbose = 1) / 5
    best_score += np.min(history.history["val_mae"]) / 5

In [ ]:
# 석식계
result2 = 0
best_score2 = 0
from sklearn.model_selection import KFold
from keras import *
from keras.layers import *
from keras.callbacks import *

i = 0
kf = KFold(n_splits = 5, shuffle = True, random_state = 82) 
for train_index, valid_index in kf.split(train_text):
    i += 1
    print(f"{i}회")
    es = EarlyStopping(patience = 20, verbose = 1)
    mc = ModelCheckpoint(f"testbest{i}.h5",save_best_only = True, verbose = 1)
    X_train, X_valid = train_text2[train_index], train_text2[valid_index] 
    X_train2,X_valid2 = new_train.iloc[train_index], new_train.iloc[valid_index]
    y_train, y_valid = train["석식계"].iloc[train_index], train["석식계"].iloc[valid_index]# 한번에 예측
    inp1 = Input((6)) 
    inp2 = Input((22))
    
    num_layer = Dense(32,activation = "elu")(inp1) # 학습
    emb = Embedding(1790,10)(inp2) # 하나의 단어당 3개의 의미 벡터
    
    branch1 = Dense(16,activation = "elu")(num_layer)
    branch2 = Flatten()(emb)
    
    concat = concatenate([branch1, branch2])
    output = Dense(1)(concat)
    model = Model([inp1,inp2],output)
    model.compile(metrics=["mae"], optimizer = "adam", loss = "mse")
    history = model.fit([X_train2,X_train],y_train,validation_data = ([X_valid2,X_valid],y_valid), epochs = 500, callbacks = [es,mc]) # 내부 베스트 점수 보기위한 history
    # rl은 이미지나 오디오에서 많이 쓰인다. 텍스트에서는 잘 쓰이지 않는다.
    model.load_weights(f"testbest{i}.h5")
    result2 += model.predict([new_test,test_text2], verbose = 1) / 5
    best_score2 += np.min(history.history["val_mae"]) / 5

In [ ]:
best_score # 114.68

In [ ]:
best_score2 # 63.09

In [ ]:
sub = pd.read_csv("../input/daconsets2/sample_submission.csv")
sub["중식계"] = result
sub["석식계"] = result2
sub

In [ ]:
sub.to_csv("sub5.csv",index=False)